# VALIDAÇÃO DOS SPLINES CÚBICOS

In [1]:
import numpy as np    
import polars as pl    
from Classes.Splines import CubicSpline as CS_MY
from scipy.interpolate import CubicSpline as CS_SP

#### FUNÇÃO DE TESTE

In [4]:
def f(t):
    return 4*t**3 + 2*t**2 + 5*t + 7

def f_dot(t):
    return 12*t**2 + 4*t + 5


#### CONSTRUÇÃO DO SPLINE

In [5]:
t_train = np.arange(0, 5.5, 0.5)
y_train = f(t_train)
cs_sp = CS_SP(t_train, y_train)
cs_my = CS_MY(t_train, y_train, f_dot(t_train[0]), f_dot(t_train[-1]), bc_type= 'clamped')

In [6]:
t_train

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

#### TESTE DO SPLINE

In [7]:
t_test = np.arange(0, 3, 0.33)
y_test = f(t_test)
y_spline_my = cs_my(t_test)
y_spline_sp = cs_sp(t_test)

#### SCIPY

In [8]:
erro_scipy = np.absolute(y_spline_sp - y_test)

### AUTORAL

In [9]:
erro_eu = np.absolute(y_spline_my - y_test)

### TABELA DE ERROS

In [21]:
df = pl.DataFrame({
    't': t_test,
    'f(t)': y_test,
    'H(t)': y_spline_sp,
    'S(t)': y_spline_my,
    '|H(t) - f(t)|': erro_scipy,
    '|S(t) - f(t)|': erro_eu
}).filter(pl.col('t')%0.5 != 0)

# Set values in scientific notation
df = df.select(pl.all().map_elements(lambda s: "{:.3e}".format(s)))

In [23]:
df.write_csv('Tabelas/Comparacao_splines.csv')
!python tably.py Tabelas/Comparacao_splines.csv > Tabelas/Comparacao_splines.tex